In [1]:
import sys
import importlib
sys.path.append('../../')

import lxml.html as html

import knutils.autoscraper as kas

In [2]:
importlib.reload(kas)

<module 'knutils.autoscraper' from '../..\\knutils\\autoscraper.py'>

In [3]:
root = html.parse('../../../DataSets/SiteSample/S1/List.html')

In [35]:
for x in root.iter():
    if x.tag == 'a' and '>>>' in x.text_content():
        print(x.attrib['href'])
        tmp = x.getparent().getparent().getparent()
        for z in tmp[0].iter():
            if z.tag == 'tr':
                print(z.text_content())

http://pejnya.net/content/chastnoe.php?news=prislannoe_837
Присланное | Раздел - частное   16 Февраля 2018   Просмотров - [52687]
http://pejnya.net/content/chastnoe.php?news=prislannoe_836
Присланное | Раздел - частное   16 Февраля 2018   Просмотров - [46732]
http://pejnya.net/content/chastnoe.php?news=prislannoe_835
Присланное | Раздел - частное   16 Февраля 2018   Просмотров - [68525]
http://pejnya.net/content/photo.php?news=podborochka_241
Подборка фоток (241) | Раздел - самое интересное   11 Февраля 2018   Просмотров - [75311]
http://pejnya.net/content/photo.php?news=vspominaya_leto_71
Вспоминаем лето (71) | Раздел - самое интересное   11 Февраля 2018   Просмотров - [98299]
http://pejnya.net/content/photo.php?news=popi_170
Попки)) Часть - 170 | Раздел - самое интересное   11 Февраля 2018   Просмотров - [95095]
http://pejnya.net/content/photo.php?news=podb_chastnih_ero_photo_665
Подборка частных эротических фоток (665) | Раздел - самое интересное   11 Февраля 2018   Просмотров - [92

In [23]:
#set labels

sample = []
target_urls = []
target_descr = []

hit_end = False
for x in root.iter():
    sample.append(x)
    if x.tag == 'a':
        url = x.attrib['href']
        if 'index.php' in url:
            hit_end = True
            
        if ('pejnya.net' in url) and ('content' in url) and not hit_end:
            target_urls.append(x)
    if x.tail is not None and '2018' in x.tail and 'Просмотров' in x.tail:
        target_descr.append(x.getparent())

In [25]:
len(target_urls)

8

In [5]:
depth_stats, tag_stats, attrk_stats, attrv_stats, attrkv_stats = kas.AutoScraper.get_stats(root)

p_max_depth = max([d for (d, n) in depth_stats if n > 1])
p_used_tags = [x for (x, n) in tag_stats if type(x) is str]
for t in ['br', 'b']:
    p_used_tags.remove(t)
t_used_attrk = ['class', 'id', 'name', 'title', 'type']
t_used_attrv_raw = [z for (k, v) in attrkv_stats.items() if k in t_used_attrk for (z, n) in v]
t_used_attrv = list(set(t_used_attrv_raw))
p_used_attrkv = [(k, v) for k in t_used_attrk for v in t_used_attrv if v in [x[0] for x in attrkv_stats[k]]]

p_context = [('cur', lambda p: p),
             ('prev-1', lambda p: kas.AutoScraper.get_sibling(p, -1)),('next-1', lambda p: kas.AutoScraper.get_sibling(p, +1)),
            ('child-0',lambda p: kas.AutoScraper.get_child(p, 0)),
             ('child-1',lambda p: kas.AutoScraper.get_child(p, 1)),
             ('child-2',lambda p: kas.AutoScraper.get_child(p, 2))]

def custom_order(clsf_name, split, split_stat):
    res = kas.SplitStat.calc_gini(clsf_name, split, split_stat)
    
    if clsf_name=='iter':
        if split.base().tag_name() == 'other':
            res *= 0.1
    elif clsf_name=='tags':
        if split.base().base().tag_name()=='other':
            res *= 0.1
            
    if clsf_name in ('tags', 'attrs'):
        if split.base().name() != 'cur':
            res *= 0.1
    
    return res

In [6]:
classifier1 = kas.ParentRelClassifier(kas.ContextClassifier(kas.TagClassifier(p_used_tags), p_context), 5)
classifier2 = kas.ParentRelClassifier(kas.ContextClassifier(kas.AttribClassifier(p_used_attrkv), p_context), 5)
classifier3 = kas.FwdIterClassifier(kas.TagClassifier(p_used_tags), 20)
fin_classifier = kas.AutoScraper({'tags':classifier1, 'attrs':classifier2, 'iter':classifier3})

In [7]:
%%time
url_scraper = fin_classifier.fit(root, target_urls, order_function=custom_order)
descr_scraper = fin_classifier.fit(root, target_descr, order_function=custom_order)
d_scrapers = {'url': url_scraper, 'descr': descr_scraper}
d_structure = {'data':{'entry':(1,), 'other': (0,0)}, 'entry':{'url':(1, 1), 'descr': (1, 1), 'other': (0, 0)}}
fin_scraper = kas.MultiScraper(d_scrapers, d_structure)

Wall time: 6.84 s


In [8]:
url_scraper.get_split_tree()

(('iter', <img> at fwd-iter-id=0, 636-0 vs 60-8),
 (('attrs', class=txt-main at context cur at rel-depth=1, 60-0 vs 0-8),
  None,
  None),
 None)

In [9]:
descr_scraper.get_split_tree()

(('attrs', class=txt-main at context cur at rel-depth=0, 679-0 vs 17-8),
 (('iter', <a> at fwd-iter-id=1, 17-0 vs 0-8), None, None),
 None)

In [9]:
urls = url_scraper.select(root)
descrs = descr_scraper.select(root)

In [10]:
res = fin_scraper.parse(root)

In [11]:
res

data(<Element td at 0x1fd1c075e58>) with {'entry': [entry(<Element table at 0x1fd1c075ea8>) with {'url': [url(<Element a at 0x1fd1c0772c8>)], 'descr': [descr(<Element td at 0x1fd1c075f48>)]}, entry(<Element table at 0x1fd1c0773b8>) with {'url': [url(<Element a at 0x1fd1c077778>)], 'descr': [descr(<Element td at 0x1fd1c077458>)]}, entry(<Element table at 0x1fd1c077868>) with {'url': [url(<Element a at 0x1fd1c077c28>)], 'descr': [descr(<Element td at 0x1fd1c077908>)]}, entry(<Element table at 0x1fd1c07a5e8>) with {'url': [url(<Element a at 0x1fd1c07a9a8>)], 'descr': [descr(<Element td at 0x1fd1c07a688>)]}, entry(<Element table at 0x1fd1c07aa98>) with {'url': [url(<Element a at 0x1fd1c07ae58>)], 'descr': [descr(<Element td at 0x1fd1c07ab38>)]}, entry(<Element table at 0x1fd1c07af48>) with {'url': [url(<Element a at 0x1fd1c07b368>)], 'descr': [descr(<Element td at 0x1fd1c07b048>)]}, entry(<Element table at 0x1fd1c07b458>) with {'url': [url(<Element a at 0x1fd1c07b818>)], 'descr': [descr(<E